In [9]:
import requests as rq
from bs4 import BeautifulSoup

url = 'https://finance.naver.com/sise/sise_deposit.naver'
data = rq.get(url)
data_html = BeautifulSoup(data.content)
parse_day = data_html.select_one('div.subtop_sise_graph2 > ul.subtop_chart_note > li > span.tah').text

print(parse_day)

  |  2023.06.08


In [10]:
import re

biz_day = re.findall('[0-9]+', parse_day)
biz_day = ''.join(biz_day)

print(biz_day)
# print(type(biz_day))
# biz_day = '20230609'

20230608


In [11]:
import requests as rq
from io import BytesIO
import pandas as pd

gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_stk = {
    'mktId' : 'STK',
    'trdDd' : biz_day,
    'money' : '1',
    'csvxls_isNo' : 'false',
    'name' : 'fileDown',
    'url' : 'dbms/MDC/STAT/standard/MDCSTAT03901'
}
headers = {'Referer' : 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}
otp_stk = rq.post(gen_otp_url, gen_otp_stk, headers=headers).text

print(otp_stk)

/IXFE7MDRlHUt2LJJQBUSkRH4sMp9miMi0E+4uzl7VgRtSksuLS7Bnxpl86F7dAOkunw9BBwugQaSjGAcH15eXnYOdUSb/E9PXMOREXl8LotBgM+EFJCxYg3zco1gIgRZqIo4cIzoURnTI8+MmkJ4m8vFLhSKmM794gFu+ThsO31lY4woqehX8j6OlXFDcfHdV4NbYo4+D2Rwcfj24VnU3Zpq3ik/Dyw3FdyOXhJkBI=


In [12]:
down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
down_sector_stk = rq.post(down_url, {'code' : otp_stk}, headers=headers)
sector_stk = pd.read_csv(BytesIO(down_sector_stk.content), encoding='EUC-KR')

sector_stk.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
0,095570,AJ네트웍스,KOSPI,서비스업,4380,-10,-0.23,205081652100
1,006840,AK홀딩스,KOSPI,기타금융,18810,-190,-1.00,249186622410
2,027410,BGF,KOSPI,기타금융,4125,30,0.73,394831762875
3,282330,BGF리테일,KOSPI,유통업,183500,-3100,-1.66,3171596751000
4,138930,BNK금융지주,KOSPI,기타금융,6940,90,1.31,2261990607240


In [13]:
gen_otp_ksq = {
    'mktId' : 'KSQ',
    'trdDd' : biz_day,
    'money' : '1',
    'csvxls_isNo' : 'false',
    'name' : 'fileDown',
    'url' : 'dbms/MDC/STAT/standard/MDCSTAT03901'
}
otp_ksq = rq.post(gen_otp_url, gen_otp_ksq, headers=headers).text

down_sector_ksq = rq.post(down_url, {'code' : otp_ksq}, headers=headers)
sector_ksq = pd.read_csv(BytesIO(down_sector_ksq.content), encoding = 'EUC-KR')

sector_ksq.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
0,060310,3S,KOSDAQ,기계·장비,2960,70,2.42,143668460320
1,054620,APS,KOSDAQ,금융,10740,-360,-3.24,219033933540
2,265520,AP시스템,KOSDAQ,반도체,21400,50,0.23,327022409400
3,211270,AP위성,KOSDAQ,통신장비,14090,-420,-2.89,212509663360
4,126600,BGF에코머티리얼즈,KOSDAQ,화학,6910,-150,-2.12,282404886740


In [14]:
krx_sector = pd.concat([sector_stk, sector_ksq]).reset_index(drop=True)
krx_sector['종목명'] = krx_sector['종목명'].str.strip()
krx_sector['기준일'] = biz_day

krx_sector.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액,기준일
0,095570,AJ네트웍스,KOSPI,서비스업,4380,-10,-0.23,205081652100,20230608
1,006840,AK홀딩스,KOSPI,기타금융,18810,-190,-1.00,249186622410,20230608
2,027410,BGF,KOSPI,기타금융,4125,30,0.73,394831762875,20230608
3,282330,BGF리테일,KOSPI,유통업,183500,-3100,-1.66,3171596751000,20230608
4,138930,BNK금융지주,KOSPI,기타금융,6940,90,1.31,2261990607240,20230608


In [18]:
import requests as rq
from io import BytesIO
import pandas as pd

gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_data = {
    'searchType' : '1',
    'mktId' : 'ALL',
    'trdDd' : biz_day,
    'csvxls_isNo' : 'false',
    'name' : 'fileDown',
    'url' : 'dbms/MDC/STAT/standard/MDCSTAT03501'
}
headers = {'Referer' : 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}
otp = rq.post(gen_otp_url, gen_otp_data, headers=headers).text

down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
krx_ind = rq.post(down_url, {'code' : otp}, headers=headers)

krx_ind = pd.read_csv(BytesIO(krx_ind.content), encoding = 'EUC-KR')
krx_ind['종목명'] = krx_ind['종목명'].str.strip()
krx_ind['기준일'] = biz_day

krx_ind

,종목코드,종목명,종가,대비,등락률,EPS,PER,선행 EPS,선행 PER,BPS,PBR,주당배당금,배당수익률,기준일
0,060310,3S,2960,70,2.42,33.0,89.70,NaN,NaN,829.0,3.57,0,0.00,20230608
1,095570,AJ네트웍스,4380,-10,-0.23,201.0,21.79,630.0,6.95,8076.0,0.54,270,6.16,20230608
2,006840,AK홀딩스,18810,-190,-1.00,NaN,NaN,NaN,NaN,41948.0,0.45,200,1.06,20230608
3,054620,APS,10740,-360,-3.24,505.0,21.27,NaN,NaN,10864.0,0.99,0,0.00,20230608
4,265520,AP시스템,21400,50,0.23,5463.0,3.92,3830.0,5.59,17980.0,1.19,270,1.26,20230608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2538,000547,흥국화재2우B,15000,370,2.53,NaN,NaN,NaN,NaN,NaN,NaN,0,0.00,20230608
2539,000545,흥국화재우,5680,-10,-0.18,NaN,NaN,NaN,NaN,NaN,NaN,0,0.00,20230608
2540,003280,흥아해운,1391,6,0.43,94.0,14.80,NaN,NaN,544.0,2.56,0,0.00,20230608
2541,037440,희림,9070,-150,-1.63,567.0,16.00,NaN,NaN,5186.0,1.75,150,1.65,20230608


In [19]:
diff = list(set(krx_sector['종목명']).symmetric_difference(set(krx_ind['종목명'])))
print(diff)

['삼성FN리츠', '잉글우드랩', '엑세스바이오', '미래에셋글로벌리츠', '한화리츠', 'NH프라임리츠', '골든센츄리', '바다로19호', 'SBI핀테크솔루션즈', '코오롱티슈진', '프레스티지바이오파마', '윙입푸드', '디앤디플랫폼리츠', '마스턴프리미어리츠', '오가닉티코스메틱', '소마젠', '케이탑리츠', '이지스밸류리츠', '신한알파리츠', '롯데리츠', '제이알글로벌리츠', '애머릿지', '크리스탈신소재', 'SK리츠', '에이리츠', '글로벌에스엠', '맵스리얼티1', '코람코에너지리츠', '맥쿼리인프라', '씨케이에이치', 'NH올원리츠', '미래에셋맵스리츠', '모두투어리츠', '신한서부티엔디리츠', '컬러레이', '이리츠코크렙', '로스웰', 'KB스타리츠', 'GRT', '엘브이엠씨홀딩스', 'JTC', 'ESR켄달스퀘어리츠', '미투젠', '헝셩그룹', '이스트아시아홀딩스', '이지스레지던스리츠', '한국ANKOR유전', '네오이뮨텍', '코람코더원리츠', '한국패러랠']


In [20]:
kor_ticker = pd.merge(krx_sector,
                      krx_ind,
                      on = krx_sector.columns.intersection(krx_ind.columns).tolist(),
                      how = 'outer'
                      )
kor_ticker.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액,기준일,EPS,PER,선행 EPS,선행 PER,BPS,PBR,주당배당금,배당수익률
0,095570,AJ네트웍스,KOSPI,서비스업,4380,-10,-0.23,205081652100,20230608,201.0,21.79,630.0,6.95,8076.0,0.54,270.0,6.16
1,006840,AK홀딩스,KOSPI,기타금융,18810,-190,-1.00,249186622410,20230608,NaN,NaN,NaN,NaN,41948.0,0.45,200.0,1.06
2,027410,BGF,KOSPI,기타금융,4125,30,0.73,394831762875,20230608,247.0,16.70,NaN,NaN,16528.0,0.25,110.0,2.67
3,282330,BGF리테일,KOSPI,유통업,183500,-3100,-1.66,3171596751000,20230608,11203.0,16.38,13735.0,13.36,55724.0,3.29,4100.0,2.23
4,138930,BNK금융지주,KOSPI,기타금융,6940,90,1.31,2261990607240,20230608,2404.0,2.89,2613.0,2.66,30468.0,0.23,625.0,9.01


In [21]:
print(kor_ticker[kor_ticker['종목명'].str.contains('스팩|제[0-9]+호')]['종목명'].values)

['엔에이치스팩19호' 'DB금융스팩10호' 'DB금융스팩8호' 'DB금융스팩9호' 'IBKS제17호스팩' 'IBKS제19호스팩'
 'IBKS제20호스팩' 'IBKS제21호스팩' 'IBKS제22호스팩' '교보11호스팩' '교보12호스팩' '교보13호스팩'
 '대신밸런스제13호스팩' '대신밸런스제14호스팩' '미래에셋드림스팩1호' '미래에셋비전스팩1호' '미래에셋비전스팩2호'
 '미래에셋비전스팩3호' '비엔케이제1호스팩' '삼성머스트스팩5호' '삼성스팩4호' '삼성스팩6호' '삼성스팩7호' '삼성스팩8호'
 '상상인제3호스팩' '신영스팩7호' '신영스팩8호' '신영스팩9호' '신한제10호스팩' '신한제7호스팩' '신한제8호스팩'
 '신한제9호스팩' '에스케이증권제8호스팩' '에이치엠씨제5호스팩' '엔에이치스팩20호' '엔에이치스팩23호' '엔에이치스팩24호'
 '엔에이치스팩25호' '엔에이치스팩26호' '엔에이치스팩27호' '엔에이치스팩28호' '유안타제10호스팩' '유안타제12호스팩'
 '유안타제13호스팩' '유안타제8호스팩' '유안타제9호스팩' '유진스팩6호' '유진스팩7호' '유진스팩8호' '유진스팩9호'
 '케이비제21호스팩' '케이비제22호스팩' '케이비제23호스팩' '키움제6호스팩' '키움제7호스팩' '키움제8호스팩'
 '하나26호스팩' '하나27호스팩' '하나금융20호스팩' '하나금융21호스팩' '하나금융22호스팩' '하나금융23호스팩'
 '하나금융24호스팩' '하나금융25호스팩' '하나머스트7호스팩' '하이제6호스팩' '하이제7호스팩' '한국제10호스팩'
 '한국제11호스팩' '한화플러스제2호스팩' '한화플러스제3호스팩']


In [22]:
print(kor_ticker[kor_ticker['종목코드'].str[-1:] != '0']['종목명'].values)

['BYC우' 'CJ4우(전환)' 'CJ씨푸드1우' 'CJ우' 'CJ제일제당 우' 'DB하이텍1우' 'DL우'
 'DL이앤씨2우(전환)' 'DL이앤씨우' 'GS우' 'JW중외제약2우B' 'JW중외제약우' 'LG생활건강우' 'LG우'
 'LG전자우' 'LG화학우' 'LX하우시스우' 'LX홀딩스1우' 'NH투자증권우' 'NPC우' 'S-Oil우' 'SK네트웍스우'
 'SK디스커버리우' 'SK우' 'SK이노베이션우' 'SK증권우' 'SK케미칼우' '계양전기우' '금강공업우' '금호건설우'
 '금호석유우' '깨끗한나라우' '남선알미우' '남양유업우' '넥센우' '넥센타이어1우B' '노루페인트우' '노루홀딩스우'
 '녹십자홀딩스2우' '대교우B' '대덕1우' '대덕전자1우' '대상우' '대상홀딩스우' '대신증권2우B' '대신증권우'
 '대원전선우' '대한제당우' '대한항공우' '덕성우' '동부건설우' '동양2우B' '동양우' '동원시스템즈우' '두산2우B'
 '두산우' '두산퓨얼셀1우' '두산퓨얼셀2우B' '롯데지주우' '롯데칠성우' '미래에셋증권2우B' '미래에셋증권우' '부국증권우'
 '삼성SDI우' '삼성물산우B' '삼성전기우' '삼성전자우' '삼성중공우' '삼성화재우' '삼양사우' '삼양홀딩스우' '서울식품우'
 '성문전자우' '성신양회우' '세방우' '솔루스첨단소재1우' '솔루스첨단소재2우B' '신영증권우' '신풍제약우'
 '아모레G3우(전환)' '아모레G우' '아모레퍼시픽우' '유안타증권우' '유유제약1우' '유유제약2우B' '유한양행우'
 '유화증권우' '일양약품우' '진흥기업2우B' '진흥기업우B' '코리아써우' '코리아써키트2우B' '코오롱글로벌우'
 '코오롱모빌리티그룹우' '코오롱우' '코오롱인더우' '크라운제과우' '크라운해태홀딩스우' '태양금속우' '태영건설우'
 '티와이홀딩스우' '하이트진로2우B' '하이트진로홀딩스우' '한국금융지주우' '한양증권우' '한진칼우' '한화3우B'
 '한화갤러리아우' '한화솔루션우' '한화우' '한화투자증권우' '현

In [23]:
print(kor_ticker[kor_ticker['종목명'].str.endswith('리츠')]['종목명'].values)

['ESR켄달스퀘어리츠' 'KB스타리츠' 'NH올원리츠' 'NH프라임리츠' 'SK리츠' '디앤디플랫폼리츠' '롯데리츠'
 '마스턴프리미어리츠' '모두투어리츠' '미래에셋글로벌리츠' '미래에셋맵스리츠' '삼성FN리츠' '신한서부티엔디리츠' '신한알파리츠'
 '에이리츠' '이지스레지던스리츠' '이지스밸류리츠' '제이알글로벌리츠' '케이탑리츠' '코람코더원리츠' '코람코에너지리츠'
 '한화리츠']


In [25]:
import numpy as np

kor_ticker['종목구분'] = np.where(kor_ticker['종목명'].str.contains('스팩|제[0-9]+호'), '스팩',
                              np.where(kor_ticker['종목코드'].str[-1:] != '0', '우선주',
                                       np.where(kor_ticker['종목명'].str.endswith('리츠'), '리츠',
                                                np.where(kor_ticker['종목명'].isin(diff), '기타',
                                                         '보통주'))))
kor_ticker = kor_ticker.reset_index(drop=True)
kor_ticker.columns = kor_ticker.columns.str.replace(' ', '')
kor_ticker = kor_ticker[['종목코드', '종목명', '시장구분', '종가', '시가총액', '기준일', 'EPS', '선행EPS', 'BPS', '주당배당금', '종목구분']]
kor_ticker = kor_ticker.replace({np.nan: None})
kor_ticker['기준일'] = pd.to_datetime(kor_ticker['기준일'])

kor_ticker.head()

,종목코드,종목명,시장구분,종가,시가총액,기준일,EPS,선행EPS,BPS,주당배당금,종목구분
0,095570,AJ네트웍스,KOSPI,4380,205081652100,2023-06-08,201.0,630.0,8076.0,270.0,보통주
1,006840,AK홀딩스,KOSPI,18810,249186622410,2023-06-08,None,None,41948.0,200.0,보통주
2,027410,BGF,KOSPI,4125,394831762875,2023-06-08,247.0,None,16528.0,110.0,보통주
3,282330,BGF리테일,KOSPI,183500,3171596751000,2023-06-08,11203.0,13735.0,55724.0,4100.0,보통주
4,138930,BNK금융지주,KOSPI,6940,2261990607240,2023-06-08,2404.0,2613.0,30468.0,625.0,보통주
